In [4]:
# source /g/data/oe9/login_script.sh
# module load agdc-py3-prod
# jupyter notebook
###
# gr Jan 18 2018
# read and write tif files that has 3 bands where 
# 1 = day of year
# 2 = NDVI in interger
# 3 = EVI in interger
# for phenology analysis
###
# stack (into time series) the tif images of NDVI EVI from MODIS
# 
from osgeo import gdal
import numpy as np
import glob
import xarray as xr
import pandas as pd

In [7]:
# read and sort file names in tif
# from 2000 to 2016
# 16 days
# band 1 day of year band 2 NDVI band 3 EVI
ndvi_tifs = glob.glob('/g/data/oe9/user/rg6346/NDVI_16D_500M/*.tif')
ndvi_tifs.sort()
ndvi_tifs
# len(ndvi_tifs)

['/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-02-18.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-03-05.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-03-21.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-04-06.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-04-22.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-05-08.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-05-24.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-06-09.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-06-25.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-07-11.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-07-27.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-08-12.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_005_NDVI_2000-08-28.tif',
 '/g/data/oe9/user/rg6346/NDVI_16D_500

In [44]:
# get the tif names in str
filenm =pd.DataFrame()
filenm ['name']=ndvi_tifs
filenm.head()

,name
0,/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_...
1,/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_...
2,/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_...
3,/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_...
4,/g/data/oe9/user/rg6346/NDVI_16D_500M/MOD13A1_...


In [60]:
# group by year 
# add date info 
# stack data into time series in each year
# write yearly stacked data to .nc files
year = np.arange(2000,2017)
for yeari in year:
    Y = list(filenm.name[filenm.name.str.contains(str(yeari))])
#     print(Y)
    ymd_ndvi = [f.rstrip('.tif')[-10:] for f in Y]
    ymd_ndvi = pd.to_datetime(ymd_ndvi, format='%Y-%m-%d')
    ymd_ndvi.name = 'time'
#     print(ymd_ndvi)
    NDVI_year = xr.concat([xr.open_rasterio(tif) for tif in Y], dim=ymd_ndvi)    
    outpath = '/g/data/oe9/user/rg6346/NDVI_stack/MOD13A1_NDVI_EVI_'+str(yeari)+'.nc'
    NDVI_year.to_netcdf(outpath, mode = 'w')
#     NDVI_all
# list(filenm.name[filenm.name.str.contains('2003')])
# name_ndvi = dirstr+'/NDVI-'+str(band)+'-'+tif.split('_')[5]

In [6]:
# sample : generate time dim to the stacked dataset from file names
ymd_ndvi = [f.rstrip('.tif')[-10:] for f in ndvi_tifs]
ymd_ndvi = pd.to_datetime(ymd_ndvi, format='%Y-%m-%d')
ymd_ndvi.name = 'time'
ymd_ndvi

DatetimeIndex(['2000-02-18', '2000-03-05', '2000-03-21', '2000-04-06',
               '2000-04-22', '2000-05-08', '2000-05-24', '2000-06-09',
               '2000-06-25', '2000-07-11',
               ...
               '2016-07-27', '2016-08-12', '2016-08-28', '2016-09-13',
               '2016-09-29', '2016-10-15', '2016-10-31', '2016-11-16',
               '2016-12-02', '2016-12-18'],
              dtype='datetime64[ns]', name='time', length=388, freq=None)

In [ ]:
%%time
NDVI_all = xr.concat([xr.open_rasterio(tif) for tif in ndvi_tifs], dim=ymd_ndvi)
NDVI_all
# sample : save stacked NDVI and EVI grouped by year in to folder(time dimention included)
